In [99]:
import warnings
warnings.simplefilter('ignore')
import pandas as pd
import numpy as np
from missingpy import KNNImputer
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import train_test_split as tts

In [54]:
train=pd.read_csv("C://Users//Neel//Desktop/train_ctrUa4K.csv")
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [55]:
train.drop(["Loan_ID"],1,inplace=True)

In [56]:
train.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [57]:
df=train.copy()

In [58]:
numerical=df.select_dtypes(include=np.number)

In [59]:
numerical.dtypes

ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
dtype: object

In [60]:
numerical['Credit_History']=numerical["Credit_History"].astype('object')

In [61]:
categorical=df.select_dtypes(exclude=np.number)

In [62]:
categorical["Credit_History"]=numerical["Credit_History"]

In [63]:
numerical.drop(['Credit_History'],1,inplace=True)

In [64]:
imputer=KNNImputer(n_neighbors=3)

In [65]:
num_cols=list(numerical)

In [66]:
numerical=pd.DataFrame(imputer.fit_transform(numerical),columns=num_cols)

In [67]:
numerical.isnull().sum()

ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
dtype: int64

In [68]:
from sklearn_pandas import CategoricalImputer

In [69]:
cat_imputer=CategoricalImputer()

In [70]:
def fill_categorical(cat_columns):
    for c in cat_columns:
        categorical[c]=cat_imputer.fit_transform(categorical[c])

In [71]:
fill_categorical(list(categorical))

In [72]:
cat_columns=list(categorical)

In [73]:
le=LabelEncoder()

In [74]:
def encode(cat_columns):
    for c in cat_columns:
        categorical[c]=le.fit_transform(categorical[c])

In [75]:
encode(["Gender","Married","Dependents","Education","Self_Employed","Loan_Status","Credit_History"])

In [78]:
categorical=pd.get_dummies(categorical)

In [81]:
num_cols=list(numerical)

In [82]:
ss=StandardScaler()

In [83]:
numerical=pd.DataFrame(ss.fit_transform(numerical),columns=num_cols)

In [85]:
df=pd.concat([numerical,categorical],1)

In [89]:
from imblearn.over_sampling import SMOTE

In [90]:
sm=SMOTE(random_state=0)

In [102]:
X=df.drop(["Loan_Status"],1)
y=train["Loan_Status"]

In [103]:
X_new,y_new=sm.fit_sample(X,y.ravel())

In [104]:
X_train,X_test,y_train,y_test=tts(X_new,y_new,test_size=0.3,random_state=0)

In [105]:
log_reg=LogisticRegression()

In [106]:
log_reg.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [107]:
y_pred_log=log_reg.predict(X_test)

In [108]:
accuracy_score(y_test,y_pred_log)

0.7559055118110236

In [109]:
print(classification_report(y_test,y_pred_log))

              precision    recall  f1-score   support

           N       0.90      0.57      0.70       127
           Y       0.69      0.94      0.79       127

    accuracy                           0.76       254
   macro avg       0.79      0.76      0.75       254
weighted avg       0.79      0.76      0.75       254

